## 1. 라이브러리 임포트

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style = 'ticks')

# 형태소 분석기 임포트
#!pip install konlpy
from konlpy.tag import Okt ; t = Okt()
import pickle

# 그래프 한글 표기
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="C:/Windows/Fonts/NanumGothic.ttf").get_name()
rc('font', family = font_name)

# word2vec 시각화
from sklearn.manifold import TSNE
import matplotlib as mpl
import gensim 
import gensim.models as g

# wordcloud
import re
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud

In [2]:
# 2. 데이터 로드

df = pd.read_csv('bluehouse_ver3.csv')
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
print(df.shape)
df.head(3)

(25398, 7)


,article_id,start,end,votes,category,title,content
0,28,2017-08-19,2017-08-26,2137,경제민주화,소액주주를 보호해주십시오,** 존경하옵는 문재인대통령님께\n저는 중국원양자원이라는 KOSPI상장사의 소액 ...
1,34,2017-08-19,2017-09-18,679,기타,『국가유공자 등 예우 및 지원에 관한법률』상「6.25전몰군경 자녀수당」의 불합리한 ...,(현황)\n우리들 아버지께서는 67여년전 북의 남침으로 조국이 위기에 처했을 때 젊...
2,37,2017-08-19,2017-09-18,415,기타,황우석박사님 연구재개 허용 촉구합니다.,국민의 70% 이상이 황우석박사님을 응원하고 지지하고 있습니다.\n*2007년 1월...


In [48]:
df['title'][180:]

180             남성들도 의무적으로 인공 자궁을 이식받을수 있는 정책이 만들어졌으면 합니다.
181                    학교에 근무하는 교육공무직 영양사들의 처우개선을 청원 드립니다.
182                           여성이 결혼 후 불러야 하는 호칭 개선을 청원합니다
183                                      학생들의 교육권을 보장해주십시오
184                            과연 LH주택공사가 무주택 서민을 위해 일하는가?
185      신재생에너지 정책과 일맥상통하는 태양광 소재산업의 선두기업인 전라북도 익산의 "넥솔...
186      배우자의 부정행위로 인한 이혼위자료및 상간자손해배상 위자료 금액을 현실적으로 증액 ...
187                     '청소노동자를 위한 휴게시설 의무설치' 법이 제정되어야합니다.
188      미세먼지 해결을 촉구합니다: 집단 미세먼지 공포증을 느끼는 국민들의 고통이 들리지 ...
189                              대통령님께 드리는 글(변호사 소재흥)/평화관련
190                          문케어, 건강보험 보장강화의 성공적 정착을 위한 제언
191                                              제사 폐지합시다.
192                             대한민국에서 진행되는 평화행사를 진행 시켜주세요
193                         평화행사가 이루어질 수 있도록 공정한 심사 부탁드립니다
194                                     창원 북면 송전탑 원천무효!!!!
195                      대한민국에서 있어지는 평화 행사에 공정한 판단 부탁드립니다.
196      하남감일지구 소음분진의 방지를 위해 방음벽이 아닌 방음터널 공사를 원합니다. (LH.

In [8]:
df_now = df[df['start'] >= '2019-12-31']
print(df_now.shape)
df_now.head(3)

(407, 7)


,article_id,start,end,votes,category,title,content
24991,584260,2019-12-31,2020-01-30,458,인권/성평등,학교폭력으로 인한 힘들어하는 한아이의 목소리를 올립니다,\r\n\t\t\t\t\t\t\t\t\t\t죄송합니다 전글은 컴퓨터상 오류가 나서 ...
24992,584261,2019-12-31,2020-01-30,377,인권/성평등,사생팬들을 더욱 엄격하게 처벌시켜주세요,\r\n\t\t\t\t\t\t\t\t\t\t최근 **** **이 엄청난 스토킹 피해...
24993,584262,2019-12-31,2020-01-30,591,교통/건축/국토,광교웰빙타운에 3호선 연장요청합니다ᆞ,\r\n\t\t\t\t\t\t\t\t\t\t경기도 수원시 광교웰빙타운에 신분당선이 ...


In [ ]:
# 제목, 내용 리스트 하나로
contents = []
for i in range(len(df)) :
    contents.append(df.iloc[i, 5] + df.iloc[i,6])

# Okt로 tokenize
token_right = t.nouns(str(contents))

# stopwords!!!!!!!!!!!!!!!!!!
stopwords=['및','지금','담당','속','내년','생','초','시기','비중','조정','날',
           '로','회','하라','료','저희','청원','집','년','원','절','달','병',
           '무비','함','맨','태','뜻','부탁','땅','개월','합','배','령','최',
           '예','염','한시','계','폐','개','겁','언제','피','너','절대','업',
           '직접','반드시','퇴','일','무','세비','사하라','누구','역','권','현',
           '킹','불','즉시','왜','때','손','외','만','힘','비소','금','용','증',
           '종','환','돈','안','경','주','검','제대로','못','난','급','이상','곳',
           '제안','여러분','이유','기','타','추가','번','실','고로','당장','제외',
           '다시','과','이하','대한','지대','위해','당한','포함','알고싶다','김',
           '대판','몰래','은혜','그것','메','맘','살','단','말','사용','구','군',
           '물','엄중','세','대해','사실','설치','또한','기재','국','알','약',
           '비','간','은', '관련', '재','정도','사적','개원','일부','무조건','정',
           '모두','통한','은','칸', '각종','위', '의','두', '양','직', '거','부',
           '제', '수', '시오', '등', '범', '고', '내', '관', '것', '를', '이', 
           '명', '저', '더', '방', '건', '남', '꼭', '좀', '중', '일어나지', '사',
           '처', '후', '자', '씨', '차', '시', '처', '입', '그', '전', '해', '왜',
           '글','제발','또','요','차라리']
no_stops = [word for word in token_right if not word in stopwords]


# 3. keyword 기반 청원 추천

In [3]:
from konlpy.tag import Okt
import re
okt=Okt()  

# 설정된 키워드
my_keyword = '남자친구 고독사 무연고자'

# 설정된 내청원 
#my_content = my['title'].iloc[0] + my['content'].iloc[0]

# contents에 keyword 삽입해 놓음
contents = []
contents.append(my_keyword)

# title 및 content 모두 추출
for i in range(len(df)) :
    contents.append(df.iloc[i, 5])

# okt로 명사만 추출한 후에 stop word로 걸러냄
for a in range(len(contents)):
    token = t.nouns((contents[a]))
    
    stopwords=['및','지금','담당','속','내년','생','초','시기','비중','조정','날','로','회','하라',
               '료','저희','청원','집','년','원','절','달','병','무비','함','맨','태','뜻','부탁',
               '땅','개월','합','배','령','최','예','염','한시','계','폐','개','겁','언제','피','너',
               '절대','업','직접','반드시','퇴','일','무','세비','사하라','누구','역','권','현','킹',
               '불','즉시','왜','때','손','외','만','힘','비소','금','용','증','종','환','돈','안',
               '경','주','검','제대로','못','난','급','이상','곳','제안','여러분','이유','기','타',
               '추가','번','실','고로','당장','제외','다시','과','이하','대한','지대','위해','당한',
               '포함','알고싶다','김','대판','몰래','은혜','그것','메','맘','살','단','말','사용','구',
               '군','물','엄중','세','대해','사실','설치','또한','기재','국','알','약','비','간','은',
               '관련','재','정도','사적','개원','일부','무조건','정','모두','통한','은','칸','각종',
               '위','의','두','양','직','거','부','제','수','시오','등','범','고','내','관','것','를',
               '이','명','저','더','방','건','남','꼭','좀','중','일어나지','사','처','후','자','씨',
               '차','시','처','입','그','전','해','왜','글','제발','또','요','차라리']
    no_stops = [word for word in token if not word in stopwords]
    contents[a] = ' '.join(t.nouns(contents[a]))


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
from sklearn.metrics.pairwise import linear_kernel

# 걸러진 명사만으로 tf_idf matrix 구성
tfidf_matrix = tfidf.fit_transform(contents)
print(tfidf_matrix.shape)

# matrix의 cosine 유사도 구하기
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# 자신의 keyword와 타 유사도 글 비교
sim_scores = list(enumerate(cosine_sim[0]))

up_zero = []

# 0 자기 자신 제외
for i in range(1,len(sim_scores)):
    if sim_scores[i][1] > 0 :
        up_zero.append(sim_scores[i])

# 유사도 순으로 sort
sim_scores_sort = sorted(up_zero, key=lambda x: x[1], reverse=True)

# 빈 dataframe 생성
df_recommend = pd.DataFrame(columns= ['article_id','start','end','votes','category','title','content'])

# 추출한 청원들을 빈 df에 추가
for i in range(len(sim_scores_sort)):
    a = sim_scores_sort[i][0]-1
    df_select = pd.DataFrame(df.iloc[a,:]).T
    df_recommend = pd.concat([df_recommend, df_select])

(25399, 13220)


NameError: name 'up_zero' is not defined

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

# 걸러진 명사만으로 tf_idf matrix 구성

tfidf_matrix = tfidf.fit_transform(contents)
print(tfidf_matrix[0])

  (0, 782)	0.48754479930561523
  (0, 3815)	0.48754479930561523
  (0, 700)	0.5019210680571425
  (0, 2113)	0.5221832808320004


In [6]:
from sklearn.metrics.pairwise import linear_kernel

# matrix의 cosine 유사도 구하기
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [7]:
# keyword와 각 content 비교

sim_scores = list(enumerate(cosine_sim[0]))

In [8]:
up_zero = []

# 0 자기 자신 제외

for i in range(1,len(sim_scores)):
    if sim_scores[i][1] > 0 :
        up_zero.append(sim_scores[i])

up_zero
        

[(9838, 0.241019618495919),
 (22986, 0.190597812256582),
 (25368, 0.6566208783303962)]

In [10]:
# 유사도 순으로 sort
sim_scores_sort = sorted(up_zero, key=lambda x: x[1], reverse=True)

# 칼럼만 가진 빈 dataframe 생성
df_recommend = pd.DataFrame(columns= ['article_id','start','end','votes','category','title','content'])

# 빈 df에 값 삽입
for i in range(len(sim_scores_sort)):
    a = sim_scores_sort[i][0]-1
    df_select = pd.DataFrame(df.iloc[a,:]).T
    df_recommend = pd.concat([df_recommend, df_select])


In [11]:
print(df_recommend.shape)
df_recommend

(3, 7)


,article_id,start,end,votes,category,title,content
25367,584636,2020-01-29,2020-02-28,326,보건복지,"고독사,무연고자 장례를 치룰수있게 도와주세요",\r\n\t\t\t\t\t\t\t\t\t\t다름이아니라 고독사를 당한 무연고자 남자...
9837,280713,2018-06-23,2018-07-23,1008,보건복지,의료기관 내부 신고자들의 신고를 경청해주세요.,"저는 의료쪽 그 중에서도 치과 종사자입니다. 그동안 많은 병원들을 보아왔고, 아직까..."
22985,582229,2019-08-23,2019-09-22,740,보건복지,"무연고, 무의무탁, 독고중장년 탈북자들의 암울한 현실과 직면한 상황을 보살펴주세요.",\r\n\t\t\t\t\t\t\t\t\t\t지난 8월 13일. 서울시 관악구에서 자...


In [21]:
df['content'][25367]

'\r\n\t\t\t\t\t\t\t\t\t\t다름이아니라 고독사를 당한 무연고자 남자친구때문에 글을올립니다\r\r\n저는 부천 **동***-* **주택에 거주했으면\r\r\n1월 22일 고인이 된  남자친구를 여자친구인 제가발견하게됐어요 전기장판위에 있어서 부폐가 많이된 상태로 충격이 아직도 가시질 않네요\r\r\n그 충격보다 우리나라 법이 참...사람을 두번죽이더라고요 고인이된 최** 오빠는 입양됐다 파양되고 아버지의 빛만 승계받아 젊은시절 어렵게자랐어요 중국집에서 일하다 , 최근까지는 배달일을 했는데요 급성심근경색으로 인해 사망하였어요\r\r\n제가 왜 DM을 보내게되었냐면요 다른건바라지도 않아요 무연고자이지만 가는길 편할수있도록 친구들과 사비로 장례만이라도 치룰수있게 도와주세요 간곡하게 부탁드릴께요 제발 젊은나이에 고생만 하다 고독사를 한 제 남자친구\r\r\n 최**에게 자비를 베풀어주세요...\r\r\n지금 **병원 장례식장 영안실에 있으며\r\r\n원미경찰서에서 담당하고있어요\r\r\n제발 장례만이라도 치룰수있게 도와주세요...\r\n\r\r\n [본 게시물의 일부 내용이 국민 청원 요건에 위배되어 관리자에 의해 수정되었습니다]\t\t\t\t\t\t\t\t\t'

In [24]:
df_recommend.to_excel('df_recommend.xlsx','w',index= False)

# 4. 내 청원 기반 청원 추천

In [3]:
# 내 청원 선택

my = pd.DataFrame(df.iloc[25251,:]).T
my.head()

,article_id,start,end,votes,category,title,content
25251,584520,2020-01-20,2020-02-19,688,인권/성평등,설명절을 앞두고 직장폐쇄라니요~ 우리의 일터에서 다시 일할수 있도록 도와주세요.,\r\n\t\t\t\t\t\t\t\t\t\t모두에게 가장 기본적인 권리\r\n\r\...


In [4]:
my.index

Int64Index([25251], dtype='int64')

In [5]:
# 내 청원 보기

my['title'].iloc[0]

'설명절을 앞두고 직장폐쇄라니요~ 우리의 일터에서 다시 일할수 있도록 도와주세요.'

In [6]:
# 내 청원 보기

my['content'].iloc[0]

'\r\n\t\t\t\t\t\t\t\t\t\t모두에게 가장 기본적인 권리\r\n\r\n\r\r\n만약 한 가정의 가장으로서 책임을 다하지 못한다면, 또 하루아침에 가족들이 차가운 길바닥에 나앉게 생겼다면 당신은 어떻게 하시겠습니까?\r\r\n이런 불행을 막기 위해 12인은 외로움 싸움을 하기 시작했습니다.\r\r\n인간이라면 당연히 챙겨야 하는 끼니는 고된 노동 후에 이미 식은 밥을 씹으며 다음 일을 고민해야 했고, 컵라면으로 허기진 배를 채우며 밤샘 노동을 준비해야 했습니다.\r\r\n온갖 녹가루와 먼지를 마시면서 몸 상하며 갖은 일을 해내고 2시간에 한번 주어지는 귀한 10분, 쉬기도 바쁘지만 100미터가량 떨어진 화장실을 가기 위해 매번 줄을 서서 기다리고도 볼일을 보지 못한 채 다시 돌아와 일을 시작합니다. 인간의 기본적인 욕구인 생리적 욕구조차 충족시키지 못하고 기계처럼 일을 해도 현장직에게 날아오는 건 막말뿐이었습니다. (또한, 현장 감독을 목적으로 CCTV를 설치한다며 그들의 감시 아래에서 숨 막히는 노동환경을 제공하려 하는 등 현장의 입장은 전혀 생각해주지 않았습니다. ) 이런 비인간적인 대우를 참으면서 저희는 매 월급 날마다 통장에 찍힌 노동의 대가에 쓴웃음 삼켰습니다. 두 어깨에는 무거운 짐을 지고 뒤를 돌아보면 가족들의 눈은 저희를 더 힘들게 했습니다. 힘든 중노동에 가까운 일을 처리하면서 (회사를 다니는 8년 동안 연차를 고작 4번을 쓰며 ) 남들 놀 때, 쉴 때 한숨 돌리지도 못하고 책임감을 가지고 일했지만 돌아오는 건 괄시, 멸시, 부당한 대우였습니다. 모회사와 너무나도 다른 자회사의 환경, 턱없이 부족한 임금, 날이 갈수록 심해지는 막말, 열악한 근무 환경을 개선하기 위해 회사에 건의도 수백 번, 하지만 그들은 단 한순간도 노동자의 입장에서 생각해주지 않았습니다. 아무리 입 아프게 설명해도 돌아오는 건 고요한 침묵뿐이었고 저희는 저희의 권리를 되찾기 위해서 노동조합을 설립했습니다. 노동조합에서는 임금에 대한 교섭과 더불어 사업장 안의

In [9]:
from konlpy.tag import Okt
import re
okt=Okt()  

# 키워드 설정
my_keyword = my['title'].iloc[0] + my['content'].iloc[0]

# contents에 keyword 삽입해 놓음
contents = []
contents.append(my_keyword)

# title 및 content 모두 추출
for i in range(len(df_now)) :
    contents.append(df_now.iloc[i, 5]+df_now.iloc[i,6])

# okt로 명사만 추출한 후에 stop word로 걸러냄
for a in range(len(contents)):
    token = t.nouns((contents[a]))
    
    stopwords=['및','지금','담당','속','내년','생','초','시기','비중','조정','날',
           '로','회','하라','료','저희','청원','집','년','원','절','달','병',
           '무비','함','맨','태','뜻','부탁','땅','개월','합','배','령','최',
           '예','염','한시','계','폐','개','겁','언제','피','너','절대','업',
           '직접','반드시','퇴','일','무','세비','사하라','누구','역','권','현',
           '킹','불','즉시','왜','때','손','외','만','힘','비소','금','용','증',
           '종','환','돈','안','경','주','검','제대로','못','난','급','이상','곳',
           '제안','여러분','이유','기','타','추가','번','실','고로','당장','제외',
           '다시','과','이하','대한','지대','위해','당한','포함','알고싶다','김',
           '대판','몰래','은혜','그것','메','맘','살','단','말','사용','구','군',
           '물','엄중','세','대해','사실','설치','또한','기재','국','알','약',
           '비','간','은', '관련', '재','정도','사적','개원','일부','무조건','정',
           '모두','통한','은','칸', '각종','위', '의','두', '양','직', '거','부',
           '제', '수', '시오', '등', '범', '고', '내', '관', '것', '를', '이', 
           '명', '저', '더', '방', '건', '남', '꼭', '좀', '중', '일어나지', '사',
           '처', '후', '자', '씨', '차', '시', '처', '입', '그', '전', '해', '왜',
           '글','제발','또','요','차라리']
    
    no_stops = [word for word in token if not word in stopwords]
    contents[a] = ' '.join(t.nouns(contents[a]))

len(contents)

408

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

# 걸러진 명사만으로 tf_idf matrix 구성

tfidf_matrix = tfidf.fit_transform(contents)
print(tfidf_matrix.shape)

(408, 11155)


In [11]:
from sklearn.metrics.pairwise import linear_kernel

# matrix의 cosine 유사도 구하기
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [12]:
# keyword와 각 content 비교

sim_scores = list(enumerate(cosine_sim[0]))

In [13]:
up_zero = []

# 0 및 자기 자신 제외

for i in range(1,len(sim_scores)):
    if sim_scores[i][1] > 0 :
        up_zero.append(sim_scores[i])

up_zero

[(1, 0.01899486515636403),
 (2, 0.007994479148571685),
 (3, 0.02764363346628487),
 (4, 0.056969937711580494),
 (5, 0.027152867348688766),
 (6, 0.03385832349886761),
 (7, 0.057725043707043205),
 (8, 0.02628241176328754),
 (9, 0.03560903105871895),
 (10, 0.02824755499137423),
 (11, 0.034179086599582374),
 (12, 0.01880860160043739),
 (13, 0.029630794306261483),
 (14, 0.014813835065271988),
 (15, 0.06735557541828598),
 (16, 0.008727944354909988),
 (17, 0.03068537753538801),
 (18, 0.028981953193015546),
 (19, 0.025156418347759658),
 (20, 0.06950523477448323),
 (21, 0.011615022453036703),
 (22, 0.06120974004839944),
 (23, 0.04146523400345497),
 (24, 0.027490290840806095),
 (25, 0.02600295826007191),
 (26, 0.02261512537466106),
 (27, 0.008793565900565115),
 (28, 0.026563901562144623),
 (29, 0.0182102989812083),
 (30, 0.0573396202787332),
 (31, 0.012792584634493098),
 (32, 0.030943351628037043),
 (33, 0.024212215846462628),
 (34, 0.03077714197546393),
 (35, 0.05029652990302254),
 (36, 0.029788

In [14]:
# 유사도 순으로 sort
sim_scores_sort = sorted(up_zero, key=lambda x: x[1], reverse=True)

# 칼럼만 가진 빈 dataframe 생성
df_recommend = pd.DataFrame(columns= ['article_id','start','end','votes','category','title','content'])

# 빈 df에 값 삽입
for i in range(len(sim_scores_sort)):
    a = sim_scores_sort[i][0]-1
    df_select = pd.DataFrame(df_now.iloc[a,:]).T
    df_recommend = pd.concat([df_recommend, df_select])


In [15]:
print(df_recommend.shape)
df_recommend

(403, 7)


,article_id,start,end,votes,category,title,content
25251,584520,2020-01-20,2020-02-19,688,인권/성평등,설명절을 앞두고 직장폐쇄라니요~ 우리의 일터에서 다시 일할수 있도록 도와주세요.,\r\n\t\t\t\t\t\t\t\t\t\t모두에게 가장 기본적인 권리\r\n\r\...
25078,584347,2020-01-07,2020-02-06,405,교통/건축/국토,국토교통부 택시운송사업 전액관리제 관리실태,"\r\n\t\t\t\t\t\t\t\t\t\t인천에 거주하는 법인택시 (**,**) ..."
25059,584328,2020-01-06,2020-02-05,1385,교통/건축/국토,택시전액관리제는 전국 법인택시 10만명이넘는 노동자를 더 힘들고 죽으라는 소리입니다..,\r\n\t\t\t\t\t\t\t\t\t\t청원합니다..저는 인천에서 법인택시를 하...
25148,584417,2020-01-13,2020-02-12,437,교통/건축/국토,서울시 택시 [ 임 금 협 정 서 ] 의 위 법을 막아주세요.,\r\n\t\t\t\t\t\t\t\t\t\t서울시 택시 한국노총산하 임금협정서를 보...
25126,584395,2020-01-10,2020-02-09,284,문화/예술/체육/언론,출판계 노동 환경은 2000년에 머물러 있습니다.,\r\n\t\t\t\t\t\t\t\t\t\t서울 마포구 합정동 및 파주 출판단지에 ...
25277,584546,2020-01-20,2020-02-19,226,기타,국회의원•고위공무원 자격 조건 및 유지를 위한 필수 봉사활동을 제안합니다.,\r\n\t\t\t\t\t\t\t\t\t\t저는 현재 육아휴직 중인 두 아이의 엄마...
25299,584568,2020-01-21,2020-02-20,325,교통/건축/국토,버스회사의 횡포 꼭 바로 잡아 주세요,\r\n\t\t\t\t\t\t\t\t\t\t12월 29일 오전 10시 40~50분경...
25192,584461,2020-01-15,2020-02-14,1554,육아/교육,"장애아동에게 정서적, 신체적, 방치.방임을 일삼은 어린이집을 엄벌해주세요.","\r\n\t\t\t\t\t\t\t\t\t\t안녕하십니까 국민 여러분, 대통령님.\r..."
25357,584626,2020-01-28,2020-02-27,202,인권/성평등,****의 강제 발령을 규탄합니다.,\r\n\t\t\t\t\t\t\t\t\t\t저는 ****(대형 마트)에 근무한지 1...
25298,584567,2020-01-21,2020-02-20,664,인권/성평등,현 이사장이 한명의 재입사를 위하여 60명을 거리로 내 몰았습니다.,\r\n\t\t\t\t\t\t\t\t\t\t너무 억울하고 또 억울하여 ..발만 동...


# 3. 2월 7일 기준 알림 내용 보기

In [36]:
df_now2 = df[df['start'] >= '2020-01-07']
df_now2.shape

(329, 7)

In [41]:
df_now2_2 = pd.read_csv('df_now2.csv')
print(df_now2_2.shape)
df_now2_2.head(3)

(355, 7)


,article_id,start,end,votes,category,title,content
0,584677,2020-02-03,2020-03-04,177,안전/환경,주 90시간 이상의 심각한 과다업무로 인해 질병 사망한 아버지의 업무상 재해승인과 ...,\r\n\t\t\t\t\t\t\t\t\t\t주당 90여 시간의 심각한 과다업무로 인...
1,584678,2020-02-03,2020-03-04,2857,교통/건축/국토,화물운송 종사자를 위한 안전운임제가 해운물류업계의 안전을 심각하게 위협하고 있습니다.,\r\n\t\t\t\t\t\t\t\t\t\t화물운송 종사자를 위한 안전운임제가 해운...
2,584679,2020-02-03,2020-03-04,228,보건복지,친권 없는 한부모도 양육비 받을수 있도록 해주세요,\r\n\t\t\t\t\t\t\t\t\t\t저는 이혼 후 친권. 양육권 없이 두아들...


In [42]:
df_now2 = pd.concat([df_now2,df_now2_2])

In [43]:
df_now2.shape

(684, 7)

In [48]:
df_now2 = df_now2.drop([25251])
df_now2.shape

(683, 7)

In [49]:
from konlpy.tag import Okt
import re
okt=Okt()  

# 키워드 설정
my_keyword = my['title'].iloc[0] + my['content'].iloc[0]

# contents에 keyword 삽입해 놓음
contents = []
contents.append(my_keyword)

# title 및 content 모두 추출
for i in range(len(df_now2)) :
    contents.append(df_now2.iloc[i, 5]+df_now2.iloc[i,6])

# okt로 명사만 추출한 후에 stop word로 걸러냄
for a in range(len(contents)):
    token = t.nouns((contents[a]))
    
    stopwords=['및','지금','담당','속','내년','생','초','시기','비중','조정','날',
           '로','회','하라','료','저희','청원','집','년','원','절','달','병',
           '무비','함','맨','태','뜻','부탁','땅','개월','합','배','령','최',
           '예','염','한시','계','폐','개','겁','언제','피','너','절대','업',
           '직접','반드시','퇴','일','무','세비','사하라','누구','역','권','현',
           '킹','불','즉시','왜','때','손','외','만','힘','비소','금','용','증',
           '종','환','돈','안','경','주','검','제대로','못','난','급','이상','곳',
           '제안','여러분','이유','기','타','추가','번','실','고로','당장','제외',
           '다시','과','이하','대한','지대','위해','당한','포함','알고싶다','김',
           '대판','몰래','은혜','그것','메','맘','살','단','말','사용','구','군',
           '물','엄중','세','대해','사실','설치','또한','기재','국','알','약',
           '비','간','은', '관련', '재','정도','사적','개원','일부','무조건','정',
           '모두','통한','은','칸', '각종','위', '의','두', '양','직', '거','부',
           '제', '수', '시오', '등', '범', '고', '내', '관', '것', '를', '이', 
           '명', '저', '더', '방', '건', '남', '꼭', '좀', '중', '일어나지', '사',
           '처', '후', '자', '씨', '차', '시', '처', '입', '그', '전', '해', '왜',
           '글','제발','또','요','차라리']
    
    no_stops = [word for word in token if not word in stopwords]
    contents[a] = ' '.join(t.nouns(contents[a]))

len(contents)

684

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

# 걸러진 명사만으로 tf_idf matrix 구성

tfidf_matrix = tfidf.fit_transform(contents)
print(tfidf_matrix.shape)

(684, 11501)


In [51]:
from sklearn.metrics.pairwise import linear_kernel

# matrix의 cosine 유사도 구하기
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [52]:
# keyword와 각 content 비교

sim_scores = list(enumerate(cosine_sim[0]))

In [53]:
up_zero = []

# 0 및 자기 자신 제외

for i in range(1,len(sim_scores)):
    if sim_scores[i][1] > 0 :
        up_zero.append(sim_scores[i])

up_zero

[(1, 0.044067563008237304),
 (2, 0.011163054262924047),
 (3, 0.009413720710834371),
 (4, 0.019266742398456414),
 (5, 0.01072523407449622),
 (6, 0.035196038476934685),
 (7, 0.04306836628288729),
 (8, 0.015213950476793248),
 (9, 0.008402340293020942),
 (10, 0.13585020992788138),
 (11, 0.03311390132559451),
 (12, 0.01885118486554653),
 (13, 0.00728405618649483),
 (14, 0.019397545098880982),
 (15, 0.02435177670097375),
 (16, 0.025005972873501016),
 (17, 0.024210489936544313),
 (18, 0.009796199371727488),
 (19, 0.019357737012541745),
 (20, 0.008835963925404854),
 (21, 0.015741626600825183),
 (22, 0.012516482355811757),
 (23, 0.02307654685935926),
 (24, 0.025139119293617732),
 (25, 0.026660217113946397),
 (26, 0.00725857582505729),
 (27, 0.00983299177996196),
 (28, 0.024142679044372523),
 (29, 0.01816416436047799),
 (30, 0.020476110673954225),
 (31, 0.04087271439437),
 (32, 0.007842351420234787),
 (33, 0.02816818316958464),
 (34, 0.001957591218905322),
 (35, 0.05517936450653983),
 (36, 0.020

In [55]:
# 유사도 순으로 sort
sim_scores_sort = sorted(up_zero, key=lambda x: x[1], reverse=True)

# 칼럼만 가진 빈 dataframe 생성
df_recommend = pd.DataFrame(columns= ['article_id','start','end','votes','category','title','content'])

# 빈 df에 값 삽입
for i in range(len(sim_scores_sort)):
    a = sim_scores_sort[i][0]-1
    df_select = pd.DataFrame(df_now2.iloc[a,:]).T
    df_recommend = pd.concat([df_recommend, df_select])

In [56]:
print(df_recommend.shape)
df_recommend

(664, 7)


,article_id,start,end,votes,category,title,content
25078,584347,2020-01-07 00:00:00,2020-02-06 00:00:00,405,교통/건축/국토,국토교통부 택시운송사업 전액관리제 관리실태,"\r\n\t\t\t\t\t\t\t\t\t\t인천에 거주하는 법인택시 (**,**) ..."
314,584991,2020-02-06,2020-03-07,158,일자리,희망의 끈,\r\n\t\t\t\t\t\t\t\t\t\t건설현장의 상습적인 임금체불.. 이로인해...
25148,584417,2020-01-13 00:00:00,2020-02-12 00:00:00,437,교통/건축/국토,서울시 택시 [ 임 금 협 정 서 ] 의 위 법을 막아주세요.,\r\n\t\t\t\t\t\t\t\t\t\t서울시 택시 한국노총산하 임금협정서를 보...
25126,584395,2020-01-10 00:00:00,2020-02-09 00:00:00,284,문화/예술/체육/언론,출판계 노동 환경은 2000년에 머물러 있습니다.,\r\n\t\t\t\t\t\t\t\t\t\t서울 마포구 합정동 및 파주 출판단지에 ...
220,584897,2020-02-04,2020-03-05,443,인권/성평등,상가사람들 살려주세요,\r\n\t\t\t\t\t\t\t\t\t\t저는 충북혁신도시 에서 식당을 운영하고 ...
25277,584546,2020-01-20 00:00:00,2020-02-19 00:00:00,226,기타,국회의원•고위공무원 자격 조건 및 유지를 위한 필수 봉사활동을 제안합니다.,\r\n\t\t\t\t\t\t\t\t\t\t저는 현재 육아휴직 중인 두 아이의 엄마...
25192,584461,2020-01-15 00:00:00,2020-02-14 00:00:00,1554,육아/교육,"장애아동에게 정서적, 신체적, 방치.방임을 일삼은 어린이집을 엄벌해주세요.","\r\n\t\t\t\t\t\t\t\t\t\t안녕하십니까 국민 여러분, 대통령님.\r..."
25299,584568,2020-01-21 00:00:00,2020-02-20 00:00:00,325,교통/건축/국토,버스회사의 횡포 꼭 바로 잡아 주세요,\r\n\t\t\t\t\t\t\t\t\t\t12월 29일 오전 10시 40~50분경...
54,584731,2020-02-03,2020-03-04,228,보건복지,신종 코로나 바이러스로 부터 우리 아빠를 지켜주세요,\r\n\t\t\t\t\t\t\t\t\t\t현 시점으로 대략 일주일 뒤면 아빠는 중...
25272,584541,2020-01-20 00:00:00,2020-02-19 00:00:00,1294,인권/성평등,'세종시 청사 어린이집 D 대학교의 갑질횡포로 속수무책 당하고 있으니 도와달라’는 ...,\r\n\t\t\t\t\t\t\t\t\t\t안녕하세요. 저는 위에서 말한 어린이집에...


In [57]:
df_recommend.to_excel('recommend_woolrim.xlsx','w',index=False)

In [61]:
d1 = df_recommend[df_recommend['article_id']==584731]
d1['content']

54    \r\n\t\t\t\t\t\t\t\t\t\t현 시점으로 대략 일주일 뒤면 아빠는 중...
Name: content, dtype: object